In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
from datetime import date
url = 'https://beta-tnsmart.rimes.int/index.php/Rainfall/daily_data'
response = requests.get(url)
if response.status_code == 200:
    print("The HTML file was imported successfully.")
    soup = BeautifulSoup(response.text, 'html')
else:
    print("An error occurred while importing the file.")
    soup = []
Da = soup.find(class_ = "panel-heading")
Date = Da.text.strip().replace("District wise observed Rainfall\n                    data on ","")
month = { "Jan" : "01", "Feb" : "02", "Mar" : "03", "Apr" : "04", "May" : "05", "Jun" : "06", "Jul" : "07",
          "Aug" : "08", "Sep" : "09", "Oct" : "10", "Nov" : "11", "Dec" : "12"}
Da = Date.replace(Date[3:6],month[Date[3:6]])
y = int(Da[6:11])
m = int(Da[3:5])
d = int(Da[0:2])
Today_Date =datetime.date(y,m,d)
if ((Today_Date == date.today())):
    print("Date match with current Date")
    table = soup.find('table', id = "data_table")
else:
    print("Date not matched")
    table = []
table_tr = table.find_all('tr')
list = [table_tr.text.strip().replace("\n\n", ",") for table_tr in table_tr][1:]
datalist = []
for i in range(len(list)):
    datalist.append(list[i].strip().replace("\n",","))
df = pd.DataFrame(datalist, columns = ['A'])
df1 = pd.DataFrame(df)
df1 = df1.A.str.split(",",expand = True)
df1 = df1.drop(columns = [6,7,8,9])
delete_rows = df1[df1[4].isna()].index
delete_rows = delete_rows.append(df1[-1:].index)
df1 = df1.drop(delete_rows,axis=0)
df1[4.1] = df1[4].str.extract('([a-zA-Z]+)')
df1[3].loc[df1[4.1].dropna().index] = df1[3].loc[df1[4.1].dropna().index] + "," + df1[4.1].dropna().values
df1[4].loc[df1[4.1].dropna().index] = df1[5].loc[df1[4.1].dropna().index]
df1 = df1.drop(columns = [0.0,5.0,4.1])
column_Name = ['dept', 'dist', 'station', 'value']
df1.columns = column_Name
df1.value = df1.value.astype(float)
df1 = df1.reset_index()
df1 = df1.drop(columns = 'index')
DL = [Da] 
for i in range (len(df1)):
    DL.append(DL[0])
C_Date = pd.DataFrame(DL,columns= ['date'])
C_Date.date = C_Date.date.astype('datetime64[ns]')
Today_Rain_Fall = df1.join(C_Date)
# Fusing the two Datas
TN_Rain_Fall_History = pd.read_csv(r"C:\Users\Arunprakash Babu\OneDrive\ドキュメント\GitHub\Portfolios\Projects\Pb5_Rainfall data analysis of TN\Total_Rain_Fall_Data.csv", index_col = 0)
TN_Rain_Fall_History.date = pd.to_datetime(TN_Rain_Fall_History.date, dayfirst = True)
if TN_Rain_Fall_History.loc[len(TN_Rain_Fall_History)-1][4] != Today_Rain_Fall.loc[0][4]:
    Total_Rain_Fall_Data = pd.concat([TN_Rain_Fall_History,Today_Rain_Fall])
    print("Data Added")
else :
    Total_Rain_Fall_Data = TN_Rain_Fall_History
    print("No Change")
Total_Rain_Fall_Data = Total_Rain_Fall_Data.reset_index(drop = True)
Total_Rain_Fall_Data.index.name = 'id'
Total_Rain_Fall_Data.to_csv(r"C:\Users\Arunprakash Babu\OneDrive\ドキュメント\GitHub\Portfolios\Projects\Pb5_Rainfall data analysis of TN\Total_Rain_Fall_Data.csv")

The HTML file was imported successfully.
Date match with current Date


C:\Users\Arunprakash Babu\AppData\Local\Temp\ipykernel_15340\893786283.py:42: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df1[3].loc[df1[4.1].dropna().index] = df1[3].loc[df1[4.1].dropna().index] + "," + df1[4.1].dropna().values
C:\Users\A

No Change
